In [7]:
import pandas as pd
import numpy as np
import ast
import os

In [14]:
calctype = 'harmonic' # 'anharmonic'
bandtype = 'Fundamental' # 'Harmonic' 'Overtone' 'CombBand'

lots_p1 = ['B971', 'B3LYP', 'wB97XD', 'M062X', 'PBE1PBE', 'TPSSh']
lots_p2 = ['B2PLYP', 'B2PLYPD']

bss_p1 = ['6-31Gs', '6-31Gss', 'pc-1', 'pcseg-1', 'AUG-pcseg-1', 'pc-2', 'pcseg-2', 'AUG-pcseg-2',  'Def2SVP', 'Def2SVPD','Def2TZVPD', 'cc-pVDZ', 'cc-pVTZ', 'AUG-cc-pVDZ']
bss_p2 = ['6-31Gs', '6-31Gss', 'pc-1', 'pcseg-1', 'AUG-pcseg-1', 'pc-2', 'pcseg-2', 'Def2SVPD', 'Def2TZVPD', 'cc-pVDZ', 'cc-pVTZ']

In [ ]:
count_ints = 0
count_other = 0

for lot in lots_p1 :
    
    output_p1_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\calculations\\final_ordered_by_lot\\{calctype}\\{lot}.csv'

    output_p1 = {'Formula' : [],
                     'SMILES' : [],
                     'Method' : [],
                     'Basis' : [],
                     'ExpFreq' : [],
                     'CCSDTFreq' : [],
                     'Mode' : [],
                     'CalcFreq' : [],
                     'Intensity' : [],
                     'Time' : [],
                     'Mode_Desc' : [],
                     'Freq_Range' : [],
                     'ScaledGlob_MC' : [],
                     'ScaledGlob_Univ' : [],
                     'ScaledFreqRange_MC' : [],
                     'ScaledFreqRange_Univ' : []}

    output_p1 = pd.DataFrame(output_p1)
        
    for bs in bss_p1 :

        calc_data_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\calculations\\final\\{lot}_{bs}.csv'
        calc_data = pd.read_csv(calc_data_path)
        
        vibfreq1295_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\vibfreq1295.csv'
        vibfreq1295 = pd.read_csv(vibfreq1295_path, encoding = 'latin1')

        all_molecules = calc_data.Formula.unique()
        all_dfs = []

        for molecule in all_molecules :

            all_dfs.append(vibfreq1295.query("Molecule == @molecule"))

        vibfreq1295_reduced = pd.concat(all_dfs, ignore_index = True)
        vibfreq1295_reduced = vibfreq1295_reduced.sort_values(['Molecule', 'SMILES', 'ExpFreq_New'])

        for mols, expfreqs, list_calc_freq, list_freq_type, list_ints in zip(calc_data['Formula'], vibfreq1295_reduced['ExpFreq_New'], calc_data['Frequencies'], calc_data['Freq_Kind'], calc_data['Intensities']) :

            try :
            
                calcfreqs = []
                intss = []

                list_calc_freq = ast.literal_eval(list_calc_freq)
                list_freq_type = ast.literal_eval(list_freq_type)
                list_ints = ast.literal_eval(list_ints)

                expfreqs = list(vibfreq1295_reduced.query(f'Molecule == "{mols}"')['ExpFreq_New'])
                print(expfreqs)

                holding = pd.DataFrame()

                for freq, freq_type, ints in zip(list_calc_freq, list_freq_type, list_ints) :
                    
                    if bandtype in freq_type :
    
                        calcfreqs.append(float(freq))
                        
                        try :
            
                            intss.append(float(ints))
                    
                        except :
                    
                            count_ints = count_ints + 1
                            print(lot, bs, mols + ' intensity too high (******)')
                        
                        total = [mols, lot, bs]
                        formula_method_basis = pd.DataFrame([total], columns = ['Formula', 'Method', 'Basis'])
                        holding = pd.concat([holding, formula_method_basis])
                        
                holding['CalcFreq'] = calcfreqs
                holding['Intensity'] = intss
                holding.sort_values(by = ['CalcFreq'], inplace = True)
                
                holding['ExpFreq'] = expfreqs
                
                output_p1 = pd.concat([output_p1, holding])
                output_p1.sort_values(by = ['Formula', 'Method', 'Basis'], inplace = True)
            
            except :
                
                count_other = count_other + 1
                print(lot, bs, mols)
    
    output_p1.to_csv(output_p1_path,index = None)

print('Intensity issues with ' + str(count_ints) + ' molecules.')
print('Other (probably degeneracy) issues with ' + str(count_other) + ' (probably C3O2) molecules.')

[1772.861]
[493.4227759, 493.4227759, 647.1340617, 907.54133, 907.54133, 1032.001419]
[199.8, 1033.7, 1070.2, 1156.5, 1335.2, 1453.3, 1474.2, 1483.5, 2844.7, 2961.1, 3003.7, 3682.5]
[439.37447, 526.07394, 587.43265, 977.68189, 1116.013, 1387.5, 2152.56, 3070.37134, 3165.2985]
[82.0, 211.0, 378.0, 434.0, 578.0, 710.0, 801.0, 809.0, 862.0, 948.0, 1024.0, 1168.0, 1173.0, 1327.0, 1355.0, 1360.0, 1414.0, 2894.0, 2898.0, 2976.0, 2976.0]
[365.5, 423.0, 600.0, 990.0, 1010.0, 1522.0]
[260.5941146, 260.5941146, 367.29555, 676.64331, 775.96139, 775.96139, 1220.33803, 1220.33803, 3032.92642]
[127.63, 341.52, 550.39, 801.47995, 1047.91, 1065.0, 1312.50599, 1338.0, 1732.09851, 1745.0, 2835.074, 2844.0]
[123.0, 420.0, 567.0, 598.0, 826.0, 1000.0, 1054.0, 1188.0, 1378.0, 1437.0, 1440.0, 1870.0, 2955.0, 3004.0, 3043.0]
[559.751]
[460.56, 828.6876, 928.0705]
[274.0, 288.0, 424.0, 435.0, 490.0, 610.0, 837.0, 853.0, 881.0, 889.0, 1015.0, 1052.0, 1086.0, 1110.0, 1128.0, 1136.0, 1217.0, 1256.0, 1291.0, 1305

[264.583167, 779.6115, 1044.8134, 1130.0, 1195.0, 1430.0, 1455.0, 1473.0, 1485.0, 1622.0218, 2820.0, 2961.0, 2985.0, 3361.0, 3427.0]
[175.5, 175.5, 253.7, 498.6, 600.1, 600.1, 810.8, 810.8, 2260.9]
[189.2, 426.02, 576.27, 912.66776, 919.29342, 935.67379, 990.77605, 1045.1922, 1170.04, 1297.86, 1377.94, 1418.9, 1442.71, 1451.04, 1653.18, 2931.46, 2954.3, 2972.3, 2991.03, 3015.0, 3091.62]
[274.0, 288.0, 424.0, 435.0, 490.0, 610.0, 837.0, 853.0, 881.0, 889.0, 1015.0, 1052.0, 1086.0, 1110.0, 1128.0, 1136.0, 1217.0, 1256.0, 1291.0, 1305.0, 1335.0, 1369.0, 1378.0, 1397.0, 1444.0, 1449.0, 1457.0, 1459.0, 2856.0, 2856.0, 2863.0, 2863.0, 2968.0, 2968.0, 2970.0, 2970.0]
[376.83, 780.0, 966.0, 1076.0, 1275.0, 1275.0, 1628.0, 1642.0, 3314.0, 3325.0, 3350.0, 3398.0]
[370.1510773, 410.0, 651.7, 792.761264, 1267.3, 1683.894022]
[893.9416]
[130.982, 373.36372, 373.36372, 434.2, 625.2, 625.2, 842.75105, 909.0, 929.3, 939.32998, 939.32998, 939.32998, 2152.0, 2154.6, 2154.6, 2155.50402, 2169.5285, 2169.5

[350.0, 418.0, 602.0, 704.0, 756.0, 785.0, 927.0, 960.0, 983.0, 1016.0, 1018.0, 1063.0, 1130.0, 1149.0, 1233.0, 1346.0, 1411.0, 1483.0, 1525.0, 1580.0, 3012.0, 3040.0, 3055.0, 3069.09]
[853.642681]
[288.9881829, 565.8310717, 602.3926346, 1033.3, 1045.7, 1258.2, 1390.7, 1578.9, 1754.1, 2853.6, 3439.5, 3563.7]
[517.87243, 1155.71299, 1362.060687]
[773.4988]
[889.07974, 1353.40466, 3577.925]
[187.0, 524.01878, 524.01878, 702.97604, 871.0, 871.0, 946.0, 946.0, 946.0, 1264.0, 1403.0, 1403.0, 2165.69, 2169.0, 2169.0, 2928.84, 2981.57, 2981.57]
[188.0, 527.0, 590.0, 602.0, 710.0, 876.0, 954.0, 2224.0, 2237.0]
[990.18213, 991.02021, 1059.20476, 1455.49505, 2971.0344, 3024.6155]
[281.5, 712.9, 759.82, 898.66, 1153.0, 1268.86, 1435.0, 2997.66, 3055.0]
[2990.9248]
[365.5, 423.0, 600.0, 990.0, 1010.0, 1522.0]
[199.8, 1033.7, 1070.2, 1156.5, 1335.2, 1453.3, 1474.2, 1483.5, 2844.7, 2961.1, 3003.7, 3682.5]
[127.63, 341.52, 550.39, 801.47995, 1047.91, 1065.0, 1312.50599, 1338.0, 1732.09851, 1745.0, 28

[519.597452, 765.355079, 1844.098709]
[2042.41851]
[725.7102]
[300.0, 641.9694, 686.5936]
[143.7434, 509.0, 767.15, 866.9629, 1107.3, 1114.04, 1352.6, 1395.2, 1429.91, 1436.64, 1747.0, 2715.77, 2923.0, 2964.44, 3014.0]
[451.32, 451.32, 1076.4921, 2318.81483]
[366.0, 642.0, 1372.0]
[376.0, 393.0, 530.0, 747.0, 808.42224, 1334.0]
[1167.25628, 1249.09468, 1500.17474, 1746.00886, 2782.4575, 2843.3256]
[737.54878, 737.54878, 853.97495, 868.35643, 868.35643, 1028.36898, 1028.36898, 1066.81, 1126.69418, 1189.21981, 1189.21981, 1191.29994, 1440.0218, 1440.0218, 1499.28218, 3019.2381, 3019.2381, 3026.7136, 3082.0, 3082.0, 3101.7525]
[408.87, 415.79, 568.19, 1109.0, 1175.0, 1414.4, 2101.5771, 3077.1, 3184.5]
[439.37447, 526.07394, 587.43265, 977.68189, 1116.013, 1387.5, 2152.56, 3070.37134, 3165.2985]
[724.35807, 1238.6246, 3609.48013]
[205.0, 525.0, 528.0]
[107.24424, 107.24424, 261.05417, 261.05417, 471.56903, 471.56903, 504.66264, 620.0, 620.0, 1154.87198, 2123.06975, 2245.374, 2296.85]
[360.

[274.0, 288.0, 424.0, 435.0, 490.0, 610.0, 837.0, 853.0, 881.0, 889.0, 1015.0, 1052.0, 1086.0, 1110.0, 1128.0, 1136.0, 1217.0, 1256.0, 1291.0, 1305.0, 1335.0, 1369.0, 1378.0, 1397.0, 1444.0, 1449.0, 1457.0, 1459.0, 2856.0, 2856.0, 2863.0, 2863.0, 2968.0, 2968.0, 2970.0, 2970.0]
[609.0, 646.35537, 749.3106, 832.0187, 859.19, 899.33009, 909.28465, 1051.75844, 1081.2906, 1091.12069, 1142.50528, 1259.0, 1329.75165, 1500.0, 1540.0, 3134.0, 3138.0, 3170.0]
[187.0, 524.01878, 524.01878, 702.97604, 871.0, 871.0, 946.0, 946.0, 946.0, 1264.0, 1403.0, 1403.0, 2165.69, 2169.0, 2169.0, 2928.84, 2981.57, 2981.57]
[211.0, 310.0, 539.0, 635.0, 675.0, 940.0, 1018.0, 1039.0, 1242.0, 1381.0, 1465.0, 2150.0, 2955.0, 2972.0, 3338.0]
[729.0, 729.0, 875.0, 961.0, 961.0, 991.0, 2206.8, 2209.0552, 2209.0552]
[123.0, 420.0, 567.0, 598.0, 826.0, 1000.0, 1054.0, 1188.0, 1378.0, 1437.0, 1440.0, 1870.0, 2955.0, 3004.0, 3043.0]
[667.37997, 667.37997, 1335.131399, 2349.142683]
[2599.04755]
[507.822011, 507.822011, 70

...

[300.0, 641.9694, 686.5936]
[175.0, 183.0, 280.0, 695.0, 742.0, 903.0, 946.0, 975.33527, 1030.0, 1315.0, 1337.0, 1427.0, 1439.0, 1442.0, 1447.0, 2919.0, 2925.0, 2970.0, 2970.0, 2997.0, 2998.0]
[458.228664, 580.303653, 648.826262, 763.154231, 886.2286661, 1303.07238, 1326.18595, 1709.567256, 3550.7]
[217.0, 265.0, 369.222808, 748.530882, 870.39575, 921.5042, 940.0, 1053.8, 1157.5, 1191.5, 1278.0, 1338.965, 1376.85008, 1391.9, 1451.0, 1461.072, 1462.488, 1471.8745, 1476.3842, 2887.0, 2887.0, 2961.7, 2967.0, 2968.2, 2968.2, 2972.6, 2976.7]
[588.7678643, 588.7678643, 1284.903334, 2223.756743]
[729.0, 729.0, 875.0, 961.0, 961.0, 991.0, 2206.8, 2209.0552, 2209.0552]
[578.0, 658.0, 663.0, 906.0, 925.0, 969.0, 1002.0, 1015.0, 1084.0, 1159.0, 1259.0, 1384.0, 1441.0, 3102.0, 3447.0]
[500.0, 888.0, 972.0, 1307.0, 1424.0, 3193.0]
[1772.861]
[893.9416]
[1080.7618, 1868.1704, 2434.4779]
[773.4988]
[493.4227759, 493.4227759, 647.1340617, 907.54133, 907.54133, 1032.001419]
[2042.41851]
[281.5, 712.9, 

[913.467, 913.467, 913.467, 974.6, 974.6, 2186.867, 2189.1865, 2189.1865, 2189.1865]
[667.37997, 667.37997, 1335.131399, 2349.142683]
[458.228664, 580.303653, 648.826262, 763.154231, 886.2286661, 1303.07238, 1326.18595, 1709.567256, 3550.7]
[1097.6251, 1391.75442, 3436.19513]
[460.56, 828.6876, 928.0705]
[175.0, 183.0, 280.0, 695.0, 742.0, 903.0, 946.0, 975.33527, 1030.0, 1315.0, 1337.0, 1427.0, 1439.0, 1442.0, 1447.0, 2919.0, 2925.0, 2970.0, 2970.0, 2997.0, 2998.0]
[500.0, 888.0, 972.0, 1307.0, 1424.0, 3193.0]
[380.310367, 679.1364116, 1166.453885]
[162.41947, 297.0923451, 512.2, 524.573925, 748.0, 888.8, 908.07196, 909.5, 972.0, 990.3, 1013.368967, 1204.2, 1277.8, 1296.2, 1380.6, 1442.2, 1596.4461, 1644.3, 2984.0, 3010.0, 3011.4, 3012.5, 3099.0, 3100.632571]
[700.93111, 1042.084054, 1103.137321]
[366.0, 642.0, 1372.0]
[130.0, 318.0, 332.0, 767.0, 925.0, 1032.0, 1166.0, 1168.0, 1207.0, 1371.0, 1443.0, 1445.0, 1454.0, 1754.0, 2943.0, 2969.0, 3012.0, 3045.0]
[417.0, 526.0, 622.0, 787.0,

[711.979, 711.979, 2096.845, 3311.476]
['C2HF' 'CH2S' 'C2H3N_1' 'CH3NO' 'C2H3Cl' 'ClNO' 'CH4' 'F2SO' 'c-C4H4O'
 'Cl2O' 'SiH3F' 'CN' 'C2H6S' 'AlCl3' 'HOCl' 'NF3' 'SO3' 'HCN' 'SiH2' 'Cl2'
 'SiH4' 'COF2' 'ONF' 'SiO' 'c-C4H8O2' 'C2Cl2' 'PF3' 'CO2' 'NCl2F' 'C2H4O2'
 'CCl2F2' 'C3H3N' 'HNO' 'SiH2Cl2' 'c-C3H6S' 'BH' 'PH3' 'NH3' 'c-C3H6'
 'C3H6' 'CH3N' 'CF4' 'ClF' 'c-C2H4S3' 'C2HCl' 'C2H2O2' 'PH' 'C2H6' 'C3H8'
 'CH' 'NSCl' 'SiH3Cl' 'F2O' 'SiHF3' 'COCl2' 'C2H4O' 'CHCl3' 'CH2Cl2'
 'C2H3N' 'COClF' 'c-C3H3NO_1' 'H2S' 'ClNO2' 'SOCl2' 'HO2' 'O3' 'CSCl2'
 'HNO3' 'CS' 'ClF3' 'P4' 'c-C3H3NO' 'NH' 'NSF' 'C3H4O' 'BH3CO' 'S2' 'H2CO'
 'C2H2O' 'S2F2' 'N2H4' 'C2H2' 'CO' 'CH6Si' 'CHF3' 'SiHCl3' 'CH5P'
 'tri-CH2' 'C4N2' 'C2H6O' 'FCN' 'HCO' 'N2O' 'c-CH2N4' 'C3H4_1' 'ClCN'
 'CSF2' 'SCl2' 'S2O' 'HNCO' 'H2O' 'sin-CH2' 'CH3Cl' 'C3H3F' 'N2F2'
 'c-C4H4N2' 'C3H3Cl' 'CH4O' 'C2N2' 'CH2O2' 'SO2' 'trans-C2H4Cl2' 'c-C4H5N'
 'CH2N2' 'HCl' 'C3H4' 'NO2' 'NHF2' 'OCS' 'CH3F' 'ClO' 'CS2' 'HOF' 'F2'
 'PN' 'SH' 'BO' 'CH5N' 'C2H5F' 

[350.0, 418.0, 602.0, 704.0, 756.0, 785.0, 927.0, 960.0, 983.0, 1016.0, 1018.0, 1063.0, 1130.0, 1149.0, 1233.0, 1346.0, 1411.0, 1483.0, 1525.0, 1580.0, 3012.0, 3040.0, 3055.0, 3069.09]
[186.0, 311.0, 451.0, 637.0, 650.0, 725.0, 908.0, 960.0, 1179.0, 1271.0, 1441.0, 2147.0, 2986.0, 3002.0, 3335.0]
[199.8, 1033.7, 1070.2, 1156.5, 1335.2, 1453.3, 1474.2, 1483.5, 2844.7, 2961.1, 3003.7, 3682.5]
[233.88016, 233.88016, 502.93192, 502.93192, 845.59392, 2157.82426, 2330.48079]
[626.1657424, 640.7251753, 1033.469965, 1109.20352, 1248.66903, 1379.05447, 1776.83215, 2942.0, 3568.9]
[517.87243, 1155.71299, 1362.060687]
[137.0, 228.0, 303.0, 712.0, 758.0, 769.0, 992.0, 1057.0, 1129.0, 1233.0, 1265.0, 1308.0, 1442.0, 1455.0, 2968.0, 2973.0, 3005.0, 3018.0]
[474.6475332, 614.01, 620.67, 692.2, 722.133012, 826.82, 864.1, 865.6, 881.54, 1016.9454, 1049.11974, 1074.62259, 1134.1, 1148.17, 1288.46, 1401.08, 1424.37803, 1472.05, 1518.55, 3118.6013, 3127.872, 3143.157, 3148.87, 3530.81134]
[408.87, 415.79,

[324.444, 324.444, 607.624, 607.624, 750.36, 2113.849097, 3339.36]
[1772.861]
[773.4988]
[219.0, 675.7, 675.7, 729.5, 977.6, 1016.9, 1091.9, 1296.2, 1428.6, 1435.2, 2304.6, 2309.0, 2936.4, 2989.8, 3002.8]
[417.0, 526.0, 622.0, 787.0, 1189.0, 1368.0]
[460.56, 828.6876, 928.0705]
[1080.7618, 1868.1704, 2434.4779]
[893.9416]
[205.0, 525.0, 528.0]
[260.0, 260.0, 944.90042, 1129.0, 1129.0, 1426.4906, 1464.0062, 1464.0062, 2161.0, 2967.0, 3013.0, 3013.0]
[331.970996, 595.8518, 1799.7316]
[366.0, 642.0, 1372.0]
[1500.818, 1565.345, 2683.9521]
[435.4, 435.4, 631.1, 631.1, 631.1, 909.1, 1283.2, 1283.2, 1283.2]
[364.71, 364.71, 919.92, 1040.79, 1040.79, 1390.0, 1448.03, 1448.03, 2266.45, 2953.92, 3040.68588, 3040.68588]
[378.0, 382.0, 556.0, 697.0, 854.0, 930.0]
[175.0, 183.0, 280.0, 695.0, 742.0, 903.0, 946.0, 975.33527, 1030.0, 1315.0, 1337.0, 1427.0, 1439.0, 1442.0, 1447.0, 2919.0, 2925.0, 2970.0, 2970.0, 2997.0, 2998.0]
[2269.22707]
[2276.20901]
[395.0, 618.57314, 720.92104, 896.57234, 942.1

In [10]:
output_p1

,Formula,SMILES,Method,Basis,ExpFreq,CCSDTFreq,Mode,CalcFreq,Intensity,Time,Mode_Desc,Freq_Range,ScaledGlob_MC,ScaledGlob_Univ,ScaledFreqRange_MC,ScaledFreqRange_Univ
0,AlCl3,NaN,TPSSh,6-31Gs,151.0000,NaN,NaN,144.057,8.195930,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,TPSSh,6-31Gs,151.0000,NaN,NaN,144.562,8.190134,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,TPSSh,6-31Gs,214.0000,NaN,NaN,193.686,39.463097,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,TPSSh,6-31Gs,375.0000,NaN,NaN,370.830,0.000084,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,TPSSh,6-31Gs,616.0000,NaN,NaN,612.915,192.478458,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,tri-CH2,NaN,TPSSh,pcseg-1,2992.0000,NaN,NaN,3030.096,1.609910,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,TPSSh,pcseg-1,3373.7000,NaN,NaN,3250.575,0.222458,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,TPSSh,pcseg-2,963.0995,NaN,NaN,956.901,15.006630,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,TPSSh,pcseg-2,2992.0000,NaN,NaN,3020.280,0.216160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
count_ints = 0
count_other = 0

for lot in lots_p2 :
    
    output_p2_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\calculations\\final_ordered_by_lot\\{calctype}\\{lot}.csv'

    output_p2 = {'Formula' : [],
                     'SMILES' : [],
                     'Method' : [],
                     'Basis' : [],
                     'ExpFreq' : [],
                     'CCSDTFreq' : [],
                     'Mode' : [],
                     'CalcFreq' : [],
                     'Intensity' : [],
                     'Time' : [],
                     'Mode_Desc' : [],
                     'Freq_Range' : [],
                     'ScaledGlob_MC' : [],
                     'ScaledGlob_Univ' : [],
                     'ScaledFreqRange_MC' : [],
                     'ScaledFreqRange_Univ' : []}

    output_p2 = pd.DataFrame(output_p2)

    for bs in bss_p2 :

        calc_data_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\calculations\\final\\{lot}_{bs}.csv'
        calc_data = pd.read_csv(calc_data_path)

        vibfreq1295_path = f'\\Users\\natia\\OneDrive\\Pulpit\\benchmark paper\\vibfreq1295.csv'
        vibfreq1295 = pd.read_csv(vibfreq1295_path, encoding = 'latin1')

        all_molecules = calc_data.Formula.unique()
        all_dfs = []

        for molecule in all_molecules :

            all_dfs.append(vibfreq1295.query("Molecule == @molecule"))

        vibfreq1295_reduced = pd.concat(all_dfs, ignore_index = True)
        vibfreq1295_reduced = vibfreq1295_reduced.sort_values(['Molecule', 'ExpFreq_New'])

        for mols, expfreqs, list_calc_freq, list_freq_type, list_ints in zip(calc_data['Formula'], vibfreq1295_reduced['ExpFreq_New'], calc_data['Frequencies'], calc_data['Freq_Kind'], calc_data['Intensities']) :
            
            try :
                
                calcfreqs = []
                intss = []

                list_calc_freq = ast.literal_eval(list_calc_freq)
                list_freq_type = ast.literal_eval(list_freq_type)
                list_ints = ast.literal_eval(list_ints)

                expfreqs = list(vibfreq1295_reduced.query(f'Molecule == "{mols}"')['ExpFreq_New'])

                holding = pd.DataFrame()

                for freq, freq_type, ints in zip(list_calc_freq, list_freq_type, list_ints) :

                    if bandtype in freq_type :

                        calcfreqs.append(float(freq))
                        
                        try :
                            
                            intss.append(float(ints))

                        except :
                            
                            count_ints = count_ints + 1
                            print(lot, bs, mols + ' intensity too high (******)')
                            
                        total = [mols, lot, bs]
                        formula_method_basis = pd.DataFrame([total], columns = ['Formula', 'Method', 'Basis'])
                        holding = pd.concat([holding, formula_method_basis])

                holding['CalcFreq'] = calcfreqs
                holding['Intensity'] = intss
                holding.sort_values(by = ['CalcFreq'], inplace = True)

                holding['ExpFreq'] = expfreqs

                output_p2 = pd.concat([output_p2, holding])
                output_p2.sort_values(by = ['Formula', 'Method', 'Basis'], inplace = True)
        
            except:
                
                count_other = count_other + 1
                print(lot,bs,mols)
            
    output_p2.to_csv(output_p2_path, index = None)
    
print('Intensity issues with ' + str(count_ints) + ' molecules.')
print('Other (probably degeneracy) issues with ' + str(count_other) + ' (probably C3O2) molecules.')

B2PLYPD pcseg-1 SiH2 intensity too high (******)
B2PLYPD pcseg-1 SiH2
Intensity issues with 1 molecules.
Other (probably degeneracy) issues with 1 (probably C3O2) molecules.


In [12]:
output_p2

,Formula,SMILES,Method,Basis,ExpFreq,CCSDTFreq,Mode,CalcFreq,Intensity,Time,Mode_Desc,Freq_Range,ScaledGlob_MC,ScaledGlob_Univ,ScaledFreqRange_MC,ScaledFreqRange_Univ
0,AlCl3,NaN,B2PLYPD,6-31Gs,151.0000,NaN,NaN,149.493,8.942225,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,B2PLYPD,6-31Gs,151.0000,NaN,NaN,149.499,8.942151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,B2PLYPD,6-31Gs,214.0000,NaN,NaN,201.257,41.569908,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,B2PLYPD,6-31Gs,375.0000,NaN,NaN,381.167,0.000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AlCl3,NaN,B2PLYPD,6-31Gs,616.0000,NaN,NaN,625.149,194.265669,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,tri-CH2,NaN,B2PLYPD,pcseg-1,2992.0000,NaN,NaN,3071.059,1.548748,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,B2PLYPD,pcseg-1,3373.7000,NaN,NaN,3284.744,0.294679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,B2PLYPD,pcseg-2,963.0995,NaN,NaN,1008.497,13.159097,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,tri-CH2,NaN,B2PLYPD,pcseg-2,2992.0000,NaN,NaN,3062.306,0.383771,NaN,NaN,NaN,NaN,NaN,NaN,NaN
